# ----------------------------------MISSING DATA----------------------------------------

### Imports

In [49]:
import pandas as pd
import numpy as np
import csv

# ------------------------------------Decline Data-----------------------------------------

In [50]:
def concatDeclineloandatacsv():
    #direct = os.path.dirname(os.getcwd()) 
    newpathdeclineloanFinal = '/Users/sonalichaudhari/Desktop/Decline'
    d1 = pd.read_csv(newpathdeclineloanFinal+'/RejectStatsA.csv', header=1, skipfooter=4, engine='python')
    d2 = pd.read_csv(newpathdeclineloanFinal+'/RejectStatsB.csv', header=1, skipfooter=4, engine='python')
    d3 = pd.read_csv(newpathdeclineloanFinal+'/RejectStatsD.csv', header=1, skipfooter=4, engine='python')
    d4 = pd.read_csv(newpathdeclineloanFinal+'/RejectStats_2016Q1.csv', header=1, skipfooter=4, engine='python')
    d5 = pd.read_csv(newpathdeclineloanFinal+'/RejectStats_2016Q2.csv', header=1, skipfooter=4, engine='python')
    d6 = pd.read_csv(newpathdeclineloanFinal+'/RejectStats_2016Q3.csv', header=1, skipfooter=4, engine='python')
    d7 = pd.read_csv(newpathdeclineloanFinal+'/RejectStats_2016Q4.csv', header=1, skipfooter=4, engine='python')
    d8 = pd.read_csv(newpathdeclineloanFinal+'/RejectStats_2017Q1.csv', header=1, skipfooter=4, engine='python')
    d9 = pd.read_csv(newpathdeclineloanFinal+'/RejectStats_2017Q2.csv', header=1, skipfooter=4, engine='python')
    d10 = pd.read_csv(newpathdeclineloanFinal+'/RejectStats_2017Q3.csv', header=1, skipfooter=4, engine='python')
    concatdata = pd.concat([d1, d2, d3, d4, d5, d6, d7, d8, d9, d10])
    return concatdata

In [51]:
concatDataReject = concatDeclineloandatacsv()

In [53]:
concatDataReject.shape

(16131432, 9)

In [52]:
concatDataReject.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [38]:
concatDataReject.isnull().sum()

Amount Requested              0
Application Date              0
Loan Title                 1295
Risk_Score              9323593
Debt-To-Income Ratio          0
Zip Code                    293
State                        22
Employment Length             0
Policy Code                 918
dtype: int64

In [6]:
def missingzipcode():
    locate = concatDataReject[['Zip Code','id', 'State']]
    Mode = concatDataReject['Zip Code'].mode()[0]
    county= list(locate['State'].unique())

    for c in county:
        temp = locate[locate['Zip Code'] == c]
        if( temp['Zip Code'].isnull().sum()==0):
            break
        else:
            if(temp.shape == temp[temp.isnull().any(axis=1)].shape ):
                temp['Zip Code'].fillna(Mode, inplace=True)
            else:
                c = temp['Zip Code'].mode()[0]
                temp['Zip Code'].fillna(c, inplace=True)
            locate.loc[locate.Id.isin(temp.Id), ['Zip Code']] = temp[['Zip Code']]
    locate['Zip Code'].fillna(locate['Zip Code'].mode()[0], inplace = True)      
    concatDataReject.loc[concatDataReject.id.isin(locate.id), ['Zip Code']] = locate[['Zip Code']]

In [43]:
# ID
concatDataReject['id'] = concatDataReject.index + 1

# DTI
concatDataReject['Debt-To-Income Ratio'] = concatDataReject['Debt-To-Income Ratio'].astype(str).map(lambda x: x.rstrip('%'))
concatDataReject['Debt-To-Income Ratio'] = concatDataReject['Debt-To-Income Ratio'].astype(float)

# Application Date
concatDataReject['Year'] = concatDataReject['Application Date'].astype(str).map(lambda x: int(x.split('-')[0]))
concatDataReject['Month'] = concatDataReject['Application Date'].astype(str).map(lambda x: x.split('-')[1])
concatDataReject['Day'] = concatDataReject['Application Date'].astype(str).map(lambda x: x.split('-')[2])
concatDataReject['Month']=concatDataReject['Month'].astype(int)
concatDataReject['Day']=concatDataReject['Day'].astype(int)

# Loan Title
concatDataReject['Loan Title'].fillna('None', inplace=True)

#Zipcode
concatDataReject['Zip Code'] = concatDataReject['Zip Code'].astype(str).map(lambda x: x.rstrip('xx'))
#missingzipcode()

# Policy Code
Mode = concatDataReject['Policy Code'].mode()[0]
concatDataReject['Policy Code'].fillna(2, inplace=True)

# Risk Score
# concatDataReject['Risk_Score'].fillna(0, inplace=True)

# State
#concatDataReject['State'] = concatDataReject['State'].fillna('NA',inplace= True)

# Employement Length
concatDataReject['Employment Length']=np.where(concatDataReject['Employment Length']=='10+ years','10',concatDataReject['Employment Length'])
concatDataReject['Employment Length']=np.where(concatDataReject['Employment Length']=='< 1 year','0',concatDataReject['Employment Length'])
concatDataReject['Employment Length']=np.where(concatDataReject['Employment Length']=='1 year','1',concatDataReject['Employment Length'])
concatDataReject['Employment Length'] = concatDataReject['Employment Length'].map(lambda x: x.rstrip('years'))
concatDataReject['Employment Length'] = concatDataReject['Employment Length'].map(lambda x: x.strip())
concatDataReject.loc[concatDataReject['Employment Length']=='n/',['Employment Length']]=0
concatDataReject['Employment Length']= concatDataReject['Employment Length'].astype(np.int64)

# Renaming
concatDataReject=concatDataReject.rename(columns = {'Amount Requested':'loan_amnt'})
concatDataReject=concatDataReject.rename(columns = {'Application Date':'application_date'})
concatDataReject=concatDataReject.rename(columns = {'Loan Title':'purpose'})
concatDataReject=concatDataReject.rename(columns = {'Zip Code':'zip_code'}) 
concatDataReject=concatDataReject.rename(columns = {'Debt-To-Income Ratio':'dti'})
concatDataReject=concatDataReject.rename(columns = {'Employment Length':'emp_length'})
concatDataReject=concatDataReject.rename(columns = {'Risk_Score':'risk_score'})
concatDataReject=concatDataReject.rename(columns = {'Application Date':'application_date'})

concatDataReject['approval']=0

/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [44]:
concatDataReject['State'].fillna('Other',inplace = True)
concatDataReject['zip_code'].fillna('NA',inplace = True)

(6807839, 9)

In [40]:
concatDataReject= concatDataReject[concatDataReject.Risk_Score.notnull()]

In [45]:
concatDataReject.isnull().sum()

loan_amnt           0
application_date    0
purpose             0
risk_score          0
dti                 0
zip_code            0
State               0
emp_length          0
Policy Code         0
id                  0
Year                0
Month               0
Day                 0
approval            0
dtype: int64

In [46]:
concatDataReject.loc[concatDataReject['zip_code']=='nan',['zip_code']]='NA'

In [47]:
concatDataReject.to_csv('/Users/sonalichaudhari/Desktop/Decline.csv')

In [48]:
concatDataReject.shape

(6807839, 14)